In [1]:
import json
import warnings
from importlib import reload

import daml_stage
import maite.protocols.image_classification as ic
import pandas as pd
from torch.utils.data import Dataset
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.models import ResNet50_Weights, resnet50
from utils import collect_metrics, collect_report_consumables, load_models_and_datasets, run_stages

reload(daml_stage)

from daml_stage import DamlTestStage

warnings.filterwarnings("ignore")

2024-06-20 17:31:55.916662: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-20 17:31:55.945179: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-20 17:31:55.945201: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-20 17:31:55.946052: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-20 17:31:55.950499: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-20 17:31:55.951263: I tensorflow/core/platform/cpu_feature_guard.cc:1

# Configure Pipeline Stages


### Panel Inputs

Based directly off of Panel mockup. Values expected to change


In [2]:
# ['CenterNet V2', 'visdrone-yolo']
model_str = "CenterNet V2"

# ['dev_train', 'dev_val', 'dev_test', 'op_train', 'op_val', 'op_test']
base_dataset_split = "dev_train"

# ['dev_train', 'dev_val', 'dev_test', 'op_train', 'op_val', 'op_test']
target_dataset_split = "dev_val"

# ['Accuracy', 'mAP']
metric = "Accuracy"

# Float
performance = 0.92

# ['Base', 'Target', 'Both']
linting = "Both"

# ['Base', 'Target', 'Both']
bias_detection = "Both"

# ['Base', 'Target', 'Both']
# feasibility = "Target"
feasibility = "Both"

# ['Base', 'Target', 'Both']
sufficiency = "Both"

# ["AE", "VAE", "VAEGMM"]
outlier_detection = "AE"

### Backend Script

A script must be used to convert the Panel config file into usable models, datasets, and TestStages


Load model


In [3]:
# Mock model
weights = ResNet50_Weights.DEFAULT
preprocess = weights.transforms()
model = resnet50(weights)
isinstance(model, ic.Model)  # Not sure where Maite Protocol check is

# Mock model with different weights
comparison_weights = ResNet50_Weights.IMAGENET1K_V1
comparison_model = resnet50(comparison_weights)
isinstance(comparison_model, ic.Model)  # Not sure where in Maite Protocol check is

True

Load datasets


In [4]:
# FMOW was too large for testing
class MaiteMNIST(Dataset):
    def __init__(self, train=True, transforms=None):
        self.dataset = MNIST("../data/", train=train, transform=transforms, download=True)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        imgs, labels = self.dataset[idx]
        return imgs, labels, {}


# ResNet50 wants 3 channels, MNIST is 1
t = transforms.Compose([transforms.Grayscale(num_output_channels=3), preprocess])
dev_dataset = MaiteMNIST(True, t)
op_dataset = MaiteMNIST(False, t)
print(f"Dev Dataset is MAITE compliant: {isinstance(dev_dataset, ic.Dataset)}")
print(f"Op Dataset is MAITE compliant: {isinstance(op_dataset, ic.Dataset)}")

Dev Dataset is MAITE compliant: True
Op Dataset is MAITE compliant: True


In [5]:
sample_config_json = '{"development": ["image properties", "visual quality", "duplicates", "outliers", "balance", "coverage", "parity", "feasibility", "sufficiency"], "operational": ["image properties", "visual quality", "duplicates", "outliers", "balance", "coverage", "parity", "drift", "out-of-distribution"]}'
config_json = json.loads(sample_config_json)

Create Stage


In [6]:
ds = DamlTestStage(config_json=config_json)
stages = [ds]

# Pipeline


In [7]:
load_models_and_datasets(
    dev_dataset=dev_dataset,
    op_dataset=op_dataset,
    model=model,
    comparison_model=comparison_model,
    target_performance=performance,
    stages=stages,
)

In [8]:
run_stages(stages=stages)

100%|██████████| 100/100 [00:01<00:00, 59.11it/s]


Saved development data


100%|██████████| 100/100 [00:01<00:00, 79.74it/s]


Saved operational data
Cache hit
{'development': {'balance': {'classes': [], 'factors': []}, 'coverage': {'coverage': 0.9}, 'duplicates': {}, 'feasibility': {'ber': 0.18, 'ber_lower': 0.095, 'max_accuracy': 0.82}, 'image properties': {}, 'outliers': {}, 'parity': {'parity': 0.25}, 'sufficiency': {'comp_model': [2, 3, 4], 'model': [1, 2, 3], 'steps': [1, 2, 3]}, 'visual quality': {}}, 'operational': {'balance': {'classes': [], 'factors': []}, 'coverage': {'coverage': 0.9}, 'drift': {'cvm': {'is_drift': False, 'pvalue': 0.6, 'val': 0.4}, 'ks': {'is_drift': True, 'pvalue': 0.8, 'val': 0.1}, 'mmd': {'is_drift': False, 'pvalue': 1.0, 'val': 0.7}}, 'duplicates': {}, 'image properties': {}, 'out-of-distribution': {}, 'outliers': {}, 'parity': {'parity': 0.25}, 'visual quality': {}}}


In [9]:
df = collect_metrics(stages=stages)
print(df)
pd.Series(df)

Returning metrics


KeyError: 'count'

In [ ]:
collect_report_consumables(stages=stages)